<a href="https://colab.research.google.com/github/Wesely/Andrlid-Animation-ClickEffect/blob/master/docs/MarketingAPI/FacebookPostPromotion_updated.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Facebook 貼文推廣 API 流程指南

本筆記本詳細說明了使用 Facebook Marketing API 推廣貼文的完整流程，包括所需的 API 呼叫序列和關鍵參數。

## 1. 導入必要的庫

首先導入需要使用的 Python 庫：

In [1]:
import requests
import json
import os
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
from IPython.display import display, HTML

# 載入所需變數
# 完整 access token，請從開發者工具弄一個 User Token 過來（用戶權杖）此用戶需要有粉專管理權，也有廣告帳戶使用權
# 或者繼續用這個「衛斯理」的Token 永遠不會到期（據說）
ACCESS_TOKEN = "EAAHlt6ZBwJk8BOxvlojvz4KME4nPzvzfBhsK4BbFKUNJytZA1dYjvtK4KEf8i7XmEZCOusF0Kb0ZA5jZAL0sHeXdKGpnhcnsvmGrw22lTh2bq0TDHWXUHdOHrBqRpKhZBEbVv9u5peg6bf4GSohOd7Wb5IV0w7CVKvGUVGx6czYRNPbj0zPOnPzDBj"

# 廣告帳戶 ID，可留空，後面有安排從 API 取得。不想要用API就先填寫
AD_ACCOUNT_ID = "839704183317453"

# 粉絲專頁 ID, 可留空，後面有安排從 API 取得。不想要用API就先填寫
PAGE_ID = "111301585143525"

# API 版本
API_VERSION = "v22.0"

# API 設定
BASE_URL = f'https://graph.facebook.com/{API_VERSION}'

## 2. 檢查 User 身份與權限

檢查 token 是否有效，並獲取有關使用者和權限的資訊

**使用的 API 端點：**
- `GET /me` - 獲取當前用戶資訊
- `GET /me/permissions` - 檢查授權權限


In [2]:
# 從環境變數取得，或在此處直接設定
SYSTEM_USER_TOKEN = ACCESS_TOKEN


# 獲取使用者可用的廣告帳戶列表
def get_user_ad_accounts():
    """獲取使用者可管理的廣告帳戶列表"""
    if not token_valid:
        print("請先確認 token 有效")
        return []

    url = f"{BASE_URL}/me/adaccounts"
    params = {
        'fields': 'id,name,account_id,account_status,currency,timezone_name,amount_spent,balance',
        'access_token': SYSTEM_USER_TOKEN
    }

    try:
        response = requests.get(url, params=params)
        response.raise_for_status()
        accounts = response.json().get('data', [])

        if not accounts:
            print("❌ 找不到任何可管理的廣告帳戶")
            return []

        print(f"✅ 找到 {len(accounts)} 個可管理的廣告帳戶")

        # 建立 DataFrame 顯示廣告帳戶
        accounts_data = []
        for i, account in enumerate(accounts):
            # 將狀態碼轉換為描述性文字
            status_codes = {
                1: '活躍',
                2: '已禁用',
                3: '已關閉',
                7: '待驗證',
                9: '暫停'
            }
            status_text = status_codes.get(account.get('account_status'), '未知')

            # 從 act_XXXXXX 格式中提取純數字 ID
            account_id = account.get('account_id', account['id'].replace('act_', ''))

            accounts_data.append({
                '序號': i+1,
                'ID': account_id,
                '名稱': account['name'],
                '狀態': status_text,
                '貨幣': account.get('currency', ''),
                '時區': account.get('timezone_name', ''),
                '已花費': account.get('amount_spent', '0'),
                '餘額': account.get('balance', '0')
            })

        df = pd.DataFrame(accounts_data)
        display(df)

        return accounts
    except Exception as e:
        print(f"❌ 獲取廣告帳戶失敗: {str(e)}")
        if hasattr(e, 'response') and e.response:
            print(f"錯誤詳情: {e.response.text}")
        return []
def verify_token_and_permissions():
    """驗證 token 和檢查擁有的權限"""
    # 檢查使用者資訊
    me_url = f"{BASE_URL}/me"
    params = {
        'fields': 'id,name',
        'access_token': SYSTEM_USER_TOKEN
    }

    try:
        response = requests.get(me_url, params=params)
        response.raise_for_status()
        user_info = response.json()

        print(f"✅ Token 有效")
        print(f"使用者 ID: {user_info['id']}")
        print(f"使用者名稱: {user_info['name']}")

        # 檢查權限
        permissions_url = f"{BASE_URL}/me/permissions"
        permissions_response = requests.get(permissions_url, params={'access_token': SYSTEM_USER_TOKEN})
        permissions_response.raise_for_status()

        permissions = permissions_response.json().get('data', [])
        permission_status = {p['permission']: p['status'] for p in permissions}

        required_permissions = [
            'ads_management',
            'ads_read',
            'pages_read_engagement',
            'pages_manage_ads',
            'business_management'
        ]

        print("\n權限檢查:")
        for perm in required_permissions:
            status = permission_status.get(perm, 'missing')
            icon = "✅" if status == 'granted' else "❌"
            print(f"{icon} {perm}: {status}")

        return True
    except Exception as e:
        print(f"❌ Token 驗證失敗: {str(e)}")
        return False

# 執行驗證
token_valid = verify_token_and_permissions()

✅ Token 有效
使用者 ID: 122112199814550567
使用者名稱: 衛斯理

權限檢查:
✅ ads_management: granted
✅ ads_read: granted
✅ pages_read_engagement: granted
✅ pages_manage_ads: granted
✅ business_management: granted


## 3. 取得廣告帳戶


In [3]:
# Token 確認有效後
if token_valid:
    # 獲取廣告帳戶
    ad_accounts = get_user_ad_accounts()

    # 選擇廣告帳戶
    if ad_accounts:
        account_index = int(input("請輸入您想使用的廣告帳戶編號 (從 1 開始): ")) - 1
        selected_account = ad_accounts[account_index]
        # 從 act_XXXXXX 格式中提取純數字 ID
        AD_ACCOUNT_ID = selected_account.get('account_id', selected_account['id'].replace('act_', ''))
        print(f"已選擇廣告帳戶: {selected_account['name']} (ID: {AD_ACCOUNT_ID})")
    else:
        print("❌ 無法繼續，請確保有可用的廣告帳戶")
        AD_ACCOUNT_ID = None

✅ 找到 5 個可管理的廣告帳戶


,序號,ID,名稱,狀態,貨幣,時區,已花費,餘額
0,1,1192699071839608,衛斯理,活躍,TWD,Asia/Taipei,0,0
1,2,839704183317453,Vannise | 溫倪詩,活躍,TWD,Asia/Shanghai,44549808,4775489
2,3,391752823792809,Vannise | 不斷電,活躍,TWD,Asia/Taipei,1837046,519756
3,4,583406577547531,Vannise | 保養先生,活躍,TWD,Asia/Taipei,236652,0
4,5,1121562915910026,Chivan Care 知凡研物,活躍,TWD,Asia/Taipei,99840,86211


請輸入您想使用的廣告帳戶編號 (從 1 開始): 2
已選擇廣告帳戶: Vannise | 溫倪詩 (ID: 839704183317453)


## 4. 獲取可用的粉絲專頁

取得使用者可管理的粉絲專頁列表，並獲取頁面 token：

**使用的 API 端點：**
- `GET /me/accounts` - 獲取用戶可管理的所有粉絲專頁


In [ ]:
def get_user_pages():
    """獲取使用者可管理的粉絲專頁列表"""
    if not token_valid:
        print("請先確認 token 有效")
        return []

    url = f"{BASE_URL}/me/accounts"
    params = {
        'fields': 'id,name,category,access_token',
        'access_token': SYSTEM_USER_TOKEN
    }

    try:
        response = requests.get(url, params=params)
        response.raise_for_status()
        pages = response.json().get('data', [])

        if not pages:
            print("❌ 找不到任何可管理的粉絲專頁")
            return []

        print(f"✅ 找到 {len(pages)} 個可管理的粉絲專頁")
        for i, page in enumerate(pages):
            print(f"{i+1}. {page['name']} (ID: {page['id']}), 類別: {page.get('category', '未知')}")

        return pages
    except Exception as e:
        print(f"❌ 獲取粉絲專頁失敗: {str(e)}")
        return []

# 獲取粉絲專頁
pages = get_user_pages()

# 選擇粉絲專頁
if pages:
    page_index = int(input("請輸入您想使用的粉絲專頁編號: ")) - 1
    selected_page = pages[page_index]
    PAGE_ID = selected_page['id']
    PAGE_ACCESS_TOKEN = selected_page['access_token']
    print(f"已選擇粉絲專頁: {selected_page['name']} (ID: {PAGE_ID})")
elif pages and PAGE_ID:
    # 從已知的 PAGE_ID 尋找對應的 token
    page_match = next((p for p in pages if p['id'] == PAGE_ID), None)
    if page_match:
        PAGE_ACCESS_TOKEN = page_match['access_token']
        print(f"已使用設定的粉絲專頁 ID: {PAGE_ID} ({page_match['name']})")
    else:
        print(f"❌ 在您的帳戶中找不到 ID 為 {PAGE_ID} 的粉絲專頁")
else:
    PAGE_ACCESS_TOKEN = None
    print("未選擇粉絲專頁，部分功能將無法使用")

✅ 找到 5 個可管理的粉絲專頁
1. Chivan Care 知凡研物 (ID: 565326239990639), 類別: 健康食品商店
2. 小八上工 (ID: 342229242316486), 類別: 軟體
3. Vannise I 不斷電 (ID: 111301585143525), 類別: 精品店
4. Vannise I 保養先生 (ID: 104439068282742), 類別: 美妝店
5. Vannise I 溫倪詩 (ID: 667274833730296), 類別: 公眾人物


## 5. 獲取粉絲專頁貼文

從選定的粉絲專頁獲取可推廣的貼文：

**使用的 API 端點：**
- `GET /{pageId}/posts` - 獲取頁面上的貼文列表，包含貼文內容、圖片和其他附件


In [ ]:
def get_page_posts(page_id, limit=10):
    """獲取粉絲專頁的貼文列表"""
    if not page_id or not PAGE_ACCESS_TOKEN:
        print("請先選擇有效的粉絲專頁")
        return []

    url = f"{BASE_URL}/{page_id}/posts"
    params = {
        'fields': 'id,message,created_time,permalink_url,full_picture,attachments{media_type,title,description,url}',
        'access_token': PAGE_ACCESS_TOKEN,
        'limit': limit
    }

    try:
        response = requests.get(url, params=params)
        response.raise_for_status()
        posts = response.json().get('data', [])

        if not posts:
            print("❌ 找不到任何貼文")
            return []

        print(f"✅ 找到 {len(posts)} 個貼文")

        # 建立 DataFrame 顯示貼文
        posts_data = []
        for post in posts:
            created_time = datetime.strptime(post['created_time'], '%Y-%m-%dT%H:%M:%S%z')
            formatted_time = created_time.strftime('%Y-%m-%d %H:%M')

            # 確定媒體類型
            media_type = 'text'
            if 'attachments' in post and post['attachments']['data']:
                media_type = post['attachments']['data'][0].get('media_type', 'unknown')

            posts_data.append({
                'ID': post['id'],
                '發布時間': formatted_time,
                '內容': post.get('message', '')[:50] + ('...' if post.get('message', '') and len(post['message']) > 50 else ''),
                '類型': media_type,
                '連結': post['permalink_url']
            })

        df = pd.DataFrame(posts_data)
        #display(df)

        return posts
    except Exception as e:
        print(f"❌ 獲取貼文失敗: {str(e)}")
        return []

# 只有在選擇了粉絲專頁後才執行
if PAGE_ID and PAGE_ACCESS_TOKEN:
    posts = get_page_posts(PAGE_ID)

    # 選擇要推廣的貼文
    if posts:
        post_index = int(input("請輸入您想要推廣的貼文索引 (從 1 開始): ")) - 1
        selected_post = posts[post_index]
        selected_post_id = selected_post['id'].split('_')[1]  # 從 PAGE_ID_POST_ID 格式中提取 POST_ID

        print(f"已選擇貼文: {selected_post.get('message', '')[:50]}... (ID: {selected_post_id})")
        # 顯示貼文圖片（如果有）
        if 'full_picture' in selected_post:
            display(HTML(f"<img src='{selected_post['full_picture']}' style='max-width: 400px; max-height: 400px;'>"))


## 6. 獲取廣告帳戶受眾包

取得廣告帳戶中可用的目標受眾包：

**使用的 API 端點：**
- `GET /act_{adAccountId}/saved_audiences` - 獲取廣告帳戶中已保存的受眾包


In [ ]:
def get_saved_audiences():
    """獲取廣告帳戶中已儲存的受眾包"""
    if not AD_ACCOUNT_ID:
        print("請先設定廣告帳戶 ID")
        return []

    url = f"{BASE_URL}/act_{AD_ACCOUNT_ID}/saved_audiences"
    params = {
        'fields': 'id,name,run_status,description,targeting',
        'access_token': SYSTEM_USER_TOKEN
    }

    try:
        response = requests.get(url, params=params)
        response.raise_for_status()
        audiences = response.json().get('data', [])

        if not audiences:
            print("❌ 找不到任何已儲存的受眾包，您需要先在廣告管理員中建立受眾包")
            return []

        print(f"✅ 找到 {len(audiences)} 個受眾包")

        # 建立 DataFrame 顯示受眾包
        audiences_data = []
        for audience in audiences:
            audiences_data.append({
                'ID': audience['id'],
                '名稱': audience['name'],
                '狀態': audience.get('run_status', 'unknown'),
                '描述': audience.get('description', '')
            })

        df = pd.DataFrame(audiences_data)
        display(df)

        return audiences
    except Exception as e:
        print(f"❌ 獲取受眾包失敗: {str(e)}")
        return []

# 獲取受眾包
audiences = get_saved_audiences()

# 選擇要使用的受眾包
selected_audience_ids = []
if audiences:
    audience_indices = input("請輸入您想要使用的受眾包索引 (可多選，以逗號分隔，從 1 開始): ")
    audience_indices = [int(idx.strip()) - 1 for idx in audience_indices.split(',')]
    selected_audience_ids = [audiences[idx]['id'] for idx in audience_indices]

    print(f"已選擇 {len(selected_audience_ids)} 個受眾包:")
    for idx in audience_indices:
        print(f"- {audiences[idx]['name']} (ID: {audiences[idx]['id']})")

## 7. 設定預算與排程

設定廣告預算和投放時間：

In [15]:
# 設定預算和排程
def setup_budget_and_schedule():
    # 設定貨幣，通常是 TWD (新台幣) 或 USD (美元)
    currency = input("請輸入預算貨幣代碼 (例如: TWD, USD): ").upper() or "TWD"

    # 設定每日預算
    daily_budget = float(input("請輸入每日預算 (例如: 300): ") or "300")

    # 設定開始日期，預設為今天
    today = datetime.now().strftime('%Y-%m-%d')
    start_date = input(f"請輸入開始日期 (格式: YYYY-MM-DD，預設: {today}): ") or today

    # 設定結束日期，預設為 7 天後
    default_end_date = (datetime.now() + timedelta(days=7)).strftime('%Y-%m-%d')
    end_date = input(f"請輸入結束日期 (格式: YYYY-MM-DD，預設: {default_end_date}): ") or default_end_date

    # 計算總預算
    start = datetime.strptime(start_date, '%Y-%m-%d')
    end = datetime.strptime(end_date, '%Y-%m-%d')
    days = (end - start).days + 1
    total_budget = daily_budget * days

    print(f"\n預算與排程摘要:")
    print(f"- 貨幣: {currency}")
    print(f"- 每日預算: {currency} {daily_budget}")
    print(f"- 開始日期: {start_date}")
    print(f"- 結束日期: {end_date}")
    print(f"- 推廣天數: {days} 天")
    print(f"- 預估總預算: {currency} {total_budget}")

    # 格式化日期為 ISO 格式，用於 API 請求
    start_time_iso = datetime.strptime(start_date, '%Y-%m-%d').strftime('%Y-%m-%dT00:00:00+0800')
    end_time_iso = datetime.strptime(end_date, '%Y-%m-%d').strftime('%Y-%m-%dT23:59:59+0800')

    return {
        'currency': currency,
        'daily_budget': daily_budget,
        'start_date': start_date,
        'end_date': end_date,
        'days': days,
        'total_budget': total_budget,
        'start_time_iso': start_time_iso,
        'end_time_iso': end_time_iso
    }

# 設定廣告目標
def setup_ad_objective():
    objectives = {
        '1': {'name': '貼文互動', 'value': 'POST_ENGAGEMENT', 'description': '增加貼文的互動，如讚、留言、分享等'},
        '2': {'name': '觸及', 'value': 'REACH', 'description': '讓更多人看到您的貼文'},
        '3': {'name': '流量', 'value': 'TRAFFIC', 'description': '將更多人引導至您的網站或應用程式'},
        '4': {'name': '影片觀看', 'value': 'VIDEO_VIEWS', 'description': '讓更多人觀看您的影片'},
        '5': {'name': '專頁讚', 'value': 'PAGE_LIKES', 'description': '為您的粉絲專頁獲取更多讚'}
    }

    print("\n請選擇廣告目標:")
    for key, obj in objectives.items():
        print(f"{key}. {obj['name']} - {obj['description']}")

    choice = input("請輸入您的選擇 (預設: 1): ") or '1'
    selected_objective = objectives[choice]

    print(f"已選擇廣告目標: {selected_objective['name']} ({selected_objective['value']})")
    return selected_objective

# 執行設定
budget_schedule = setup_budget_and_schedule()
ad_objective = setup_ad_objective()

請輸入預算貨幣代碼 (例如: TWD, USD): TWD
請輸入每日預算 (例如: 300): 300
請輸入開始日期 (格式: YYYY-MM-DD，預設: 2025-04-15): 2024-04-15
請輸入結束日期 (格式: YYYY-MM-DD，預設: 2025-04-22): 2024-04-17

預算與排程摘要:
- 貨幣: TWD
- 每日預算: TWD 300.0
- 開始日期: 2024-04-15
- 結束日期: 2024-04-17
- 推廣天數: 3 天
- 預估總預算: TWD 900.0

請選擇廣告目標:
1. 貼文互動 - 增加貼文的互動，如讚、留言、分享等
2. 觸及 - 讓更多人看到您的貼文
3. 流量 - 將更多人引導至您的網站或應用程式
4. 影片觀看 - 讓更多人觀看您的影片
5. 專頁讚 - 為您的粉絲專頁獲取更多讚
請輸入您的選擇 (預設: 1): 1
已選擇廣告目標: 貼文互動 (POST_ENGAGEMENT)


## 8. 建立廣告活動

創建廣告活動作為所有廣告的容器:

**使用的 API 端點：**
- `POST /act_{adAccountId}/campaigns` - 創建新的廣告活動，需要指定目標、狀態等參數


In [1]:

def create_campaign(ad_account_id, name, objective, status='PAUSED'):
    """建立廣告活動"""
    url = f"{BASE_URL}/act_{ad_account_id}/campaigns"

    data = {
        'name': name,
        'objective': objective,
        'status': status,
        'special_ad_categories': ['NONE'],
        'access_token': SYSTEM_USER_TOKEN,
        'campaign_optimization_type': 'NONE',  # 修改為 NONE
        'bid_strategy': 'LOWEST_COST_WITHOUT_CAP'
    }

    # 接收完整的 API 錯誤訊息
    if 'POST_ENGAGEMENT' in objective:
        data['promoted_object'] = json.dumps({
            'page_id': PAGE_ID
        })

    # 顯示請求參數（隱藏 token）
    display_data = data.copy()
    if 'access_token' in display_data:
        display_data['access_token'] = display_data['access_token']
    print(f"📤 發送請求到 {url}")
    print(f"📝 請求參數: {json.dumps(display_data, indent=2, ensure_ascii=False)}")
    # 重複一下權限檢查
    verify_token_and_permissions()

    try:
        response = requests.post(url, data=data)
        print(f"📥 API 回應代碼: {response.status_code}")
        print(f"📥 API 回應內容: {response.text}")
        response.raise_for_status()
        result = response.json()
        campaign_id = result['id']
        print(f"✅ 廣告活動建立成功，ID: {campaign_id}")
        return campaign_id
    except Exception as e:
        print(f"❌ 建立廣告活動失敗: {str(e)}")
        if hasattr(e, 'response') and e.response:
            print(f"錯誤詳情: {e.response.text}")
        return None

## 執行呼叫
campaign_name = f"推廣貼文 - {selected_post.get('message', '')[:10] or '貼文'}..."
campaign_id = create_campaign(AD_ACCOUNT_ID, campaign_name, ad_objective['value'])


NameError: name 'selected_post' is not defined

## 9. 建立廣告組合

為每個選定的受眾建立廣告組合:

**使用的 API 端點：**
- `POST /act_{adAccountId}/adsets` - 創建廣告組合，需要指定廣告活動ID、預算、排程和目標受眾


In [ ]:
def create_adset(ad_account_id, campaign_id, name, audience_id, daily_budget, start_time, end_time, optimization_goal, status='PAUSED'):
    """建立廣告組合"""
    url = f"{BASE_URL}/act_{ad_account_id}/adsets"

    # 轉換預算為最小貨幣單位 (cents)
    budget_in_cents = int(daily_budget * 100)

    data = {
        'campaign_id': campaign_id,
        'name': name,
        'status': status,
        'billing_event': 'IMPRESSIONS',
        'optimization_goal': optimization_goal,
        'bid_amount': 100,  # 出價金額，通常以最小貨幣單位計算
        'daily_budget': budget_in_cents,
        'start_time': start_time,
        'end_time': end_time,
        'targeting': json.dumps({
            # 'saved_audience_id': audience_id ## 將來啟用，測試時先直接用寫死
            "geo_locations": {
                "countries": ["TW"]
            }
        }),
        'access_token': SYSTEM_USER_TOKEN
    }

    try:
        response = requests.post(url, data=data)
        response.raise_for_status()
        result = response.json()
        adset_id = result['id']
        print(f"✅ 廣告組合建立成功，ID: {adset_id}")
        return adset_id
    except Exception as e:
        print(f"❌ 建立廣告組合失敗: {str(e)}")
        if hasattr(e, 'response') and e.response:
            print(f"錯誤詳情: {e.response.text}")
        return None

# 這裡只顯示代碼，不實際執行
# adset_ids = []
# for audience_id in selected_audience_ids:
#     audience = next((a for a in audiences if a['id'] == audience_id), {'name': '未知受眾'})
#     adset_name = f"推廣貼文 - {audience['name']}"
#     adset_id = create_adset(
#         AD_ACCOUNT_ID,
#         campaign_id,
#         adset_name,
#         audience_id,
#         budget_schedule['daily_budget'],
#         budget_schedule['start_time_iso'],
#         budget_schedule['end_time_iso'],
#         ad_objective['value']
#     )
#     if adset_id:
#         adset_ids.append(adset_id)

## 10. 建立廣告

為每個廣告組合建立廣告，指定要推廣的貼文:

**使用的 API 端點：**
- `POST /act_{adAccountId}/ads` - 創建廣告，需要指定廣告組合ID和創意內容


In [ ]:
def create_ad(ad_account_id, adset_id, name, page_id, post_id):
    """建立廣告"""
    url = f"{BASE_URL}/act_{ad_account_id}/ads"

    data = {
        'name': name,
        'adset_id': adset_id,
        'status': 'ACTIVE',
        'creative': json.dumps({
            'object_story_id': f"{page_id}_{post_id}"
        }),
        'access_token': SYSTEM_USER_TOKEN
    }

    try:
        response = requests.post(url, data=data)
        response.raise_for_status()
        result = response.json()
        ad_id = result['id']
        print(f"✅ 廣告建立成功，ID: {ad_id}")
        return ad_id
    except Exception as e:
        print(f"❌ 建立廣告失敗: {str(e)}")
        if hasattr(e, 'response') and e.response:
            print(f"錯誤詳情: {e.response.text}")
        return None

# 這裡只顯示代碼，不實際執行
# ad_ids = []
# for adset_id in adset_ids:
#     ad_name = f"貼文推廣 - {datetime.now().strftime('%Y-%m-%d')}"
#     ad_id = create_ad(
#         AD_ACCOUNT_ID,
#         adset_id,
#         ad_name,
#         PAGE_ID,
#         selected_post_id
#     )
#     if ad_id:
#         ad_ids.append(ad_id)

## 11. 啟用廣告活動

在所有元素都建立好後，啟用廣告活動和廣告組合:

**使用的 API 端點：**
- `POST /{campaignId}` - 更新廣告活動狀態（例如從 PAUSED 改為 ACTIVE）
- `POST /{adsetId}` - 更新廣告組合狀態


In [ ]:
def update_campaign_status(campaign_id, status):
    """更新廣告活動狀態"""
    url = f"{BASE_URL}/{campaign_id}"

    data = {
        'status': status,
        'access_token': SYSTEM_USER_TOKEN
    }

    try:
        response = requests.post(url, data=data)
        response.raise_for_status()
        print(f"✅ 廣告活動狀態已更新為: {status}")
        return True
    except Exception as e:
        print(f"❌ 更新廣告活動狀態失敗: {str(e)}")
        return False

def update_adset_status(adset_id, status):
    """更新廣告組合狀態"""
    url = f"{BASE_URL}/{adset_id}"

    data = {
        'status': status,
        'access_token': SYSTEM_USER_TOKEN
    }

    try:
        response = requests.post(url, data=data)
        response.raise_for_status()
        print(f"✅ 廣告組合狀態已更新為: {status}")
        return True
    except Exception as e:
        print(f"❌ 更新廣告組合狀態失敗: {str(e)}")
        return False

# 這裡只顯示代碼，不實際執行
# # 啟用廣告活動
# update_campaign_status(campaign_id, 'ACTIVE')
#
# # 啟用所有廣告組合
# for adset_id in adset_ids:
#     update_adset_status(adset_id, 'ACTIVE')

## 12. 完整的貼文推廣流程

將所有步驟整合到一個完整的函數中:

**使用的 API 端點順序：**
1. `POST /act_{adAccountId}/campaigns` - 創建廣告活動
2. `POST /act_{adAccountId}/adsets` - 為每個受眾創建廣告組合
3. `POST /act_{adAccountId}/ads` - 為每個廣告組合創建廣告
4. `POST /{campaignId}` - 啟用廣告活動
5. `POST /{adsetId}` - 啟用廣告組合


In [ ]:
def promote_facebook_post(ad_account_id, page_id, post_id, post_title, audience_ids, budget_info, ad_objective):
    """完整的貼文推廣流程"""
    print("\n開始執行貼文推廣流程...\n")
    results = {
        'status': 'error',
        'campaign_id': None,
        'adset_ids': [],
        'ad_ids': []
    }

    try:
        # 1. 建立廣告活動
        campaign_name = f"推廣貼文 - {post_title[:30]}..."
        campaign_id = create_campaign(ad_account_id, campaign_name, ad_objective, 'PAUSED')

        if not campaign_id:
            return results

        results['campaign_id'] = campaign_id

        # 2. 建立廣告組合
        adset_ids = []
        for audience_id in audience_ids:
            audience = next((a for a in audiences if a['id'] == audience_id), {'name': '未知受眾'})
            adset_name = f"推廣貼文 - {audience['name']}"
            adset_id = create_adset(
                ad_account_id,
                campaign_id,
                adset_name,
                audience_id,
                budget_info['daily_budget'],
                budget_info['start_time_iso'],
                budget_info['end_time_iso'],
                ad_objective,
                'PAUSED'
            )
            if adset_id:
                adset_ids.append(adset_id)

        if not adset_ids:
            return results

        results['adset_ids'] = adset_ids

        # 3. 建立廣告
        ad_ids = []
        for adset_id in adset_ids:
            ad_name = f"貼文推廣 - {datetime.now().strftime('%Y-%m-%d')}"
            ad_id = create_ad(
                ad_account_id,
                adset_id,
                ad_name,
                page_id,
                post_id
            )
            if ad_id:
                ad_ids.append(ad_id)

        if not ad_ids:
            return results

        results['ad_ids'] = ad_ids

        # 4. 啟用廣告活動和廣告組合
        update_campaign_status(campaign_id, 'ACTIVE')

        for adset_id in adset_ids:
            update_adset_status(adset_id, 'ACTIVE')

        # 5. 完成
        results['status'] = 'success'
        print("\n✅ 貼文推廣流程完成！")
        print(f"- 廣告活動 ID: {campaign_id}")
        print(f"- 廣告組合數量: {len(adset_ids)}")
        print(f"- 廣告數量: {len(ad_ids)}")

        return results

    except Exception as e:
        print(f"❌ 推廣流程中發生錯誤: {str(e)}")
        return results

# 執行完整推廣流程 (取消註解以實際執行)
'''
if all([AD_ACCOUNT_ID, PAGE_ID, selected_post_id, selected_audience_ids, budget_schedule, ad_objective]):
    confirmation = input("\n準備開始推廣貼文。確認要繼續嗎？(y/n): ")
    if confirmation.lower() == 'y':
        promotion_result = promote_facebook_post(
            AD_ACCOUNT_ID,
            PAGE_ID,
            selected_post_id,
            selected_post.get('message', '')[:30] or '貼文',
            selected_audience_ids,
            budget_schedule,
            ad_objective['value']
        )

        if promotion_result['status'] == 'success':
            print("\n🎉 推廣已成功建立！您可以在 Facebook 廣告管理員中查看和管理您的廣告。")
        else:
            print("\n⚠️ 推廣過程中發生錯誤，請檢查錯誤訊息並重試。")
    else:
        print("推廣已取消。")
else:
    print("❌ 缺少必要的資訊，無法執行推廣流程。請先完成所有前置步驟。")
'''

## 13. API 呼叫摘要

以下是整個貼文推廣流程中使用的 Meta API 呼叫摘要：

### 前置驗證與資料獲取

1. **驗證 Token**
   - `GET /me`
   - `GET /me/permissions`

2. **獲取粉絲專頁**
   - `GET /me/accounts`

3. **獲取貼文**
   - `GET /{pageId}/posts`

4. **獲取受眾包**
   - `GET /act_{adAccountId}/saved_audiences`

### 廣告創建流程

5. **建立廣告活動**
   - `POST /act_{adAccountId}/campaigns`

6. **建立廣告組合** (為每個受眾建立)
   - `POST /act_{adAccountId}/adsets`

7. **建立廣告** (為每個廣告組合建立)
   - `POST /act_{adAccountId}/ads`

8. **啟用廣告活動和廣告組合**
   - `POST /{campaignId}` (更新狀態)
   - `POST /{adsetId}` (更新狀態)

## 14. 小結

本筆記本展示了使用 Meta Marketing API 推廣 Facebook 貼文的完整流程，涵蓋了以下關鍵步驟：

1. **前置準備**：驗證 token 和權限
2. **選擇來源**：選擇粉絲專頁和要推廣的貼文
3. **設定目標**：選擇廣告目標和受眾包
4. **預算與排程**：設定預算、投放時間
5. **建立廣告**：依序建立廣告活動、廣告組合和廣告

請注意，在實際使用時，您需要：
- 擁有足夠的 Facebook API 權限
- 有權限存取的廣告帳戶和粉絲專頁
- 已建立的受眾包
- 符合 Facebook 廣告政策的內容

完整的 API 文件請參考 [Facebook Marketing API 文檔](https://developers.facebook.com/docs/marketing-apis)